![ERA-LOGO](data/logo.png)

## Initialise Python Environment
Run the cell bellow to get everything setup and ready to go! \
This can take some time, so sit back and wait a few minutes...

In [ ]:
from ERA.requirements import install; await install();
from ERA.software import *
output_view = widgets.Output()
output_view

<br />
Run this cell to clear the Output View:

In [ ]:
output_view.clear_output()

<br />

## Upload a file:
Run the cell below, then click "Upload" and select your data file.

In [ ]:
uploader = FileUploader()

## Processing the data:
Before we can analyse the data, we first need to check what was recorded.
Run the following code chunk to retrieve this information from the UVfits file.

In [ ]:
#uploader.write_file()
data = ERA.DataFile("data/example.uvfits",print_metadata=True)

## Figure: UV Coverage

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_uv_coverage() )

## Figure: Visibilities vs Baseline Length

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_vis_v_baseline() )

## Figure: All sky image

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_all_sky_image() )

## Figure: Image vs Baseline Length (Increasing Resolution)

In [ ]:
# Generate data for each animation frame
Nframes = 16
data = np.zeros([npix,npix,Nframes])
uvmax = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)

for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]
  data[:,:,n] = gen_image(select=select)

# --- Define animation frames ---
frames = []
slider_steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax *(c/freq_center) # (metres)

  # --- define data for this frame ---
  frames.append(
    go.Heatmap(
      name = "",
      z = data[:,:,n],
      x = l_axis,
      y = m_axis,
      colorscale = colorscale,
      visible = False #All frames not visible by default (We will manually enable frame 0 later)
    )
  )

  # --- Define plot title and slider related stuff for this frame ---
  slider_step = {
      "method": "restyle", #"update",
      "label": np.round(uv_cutoff,2),
      "args": [
                {"visible": [False] * Nframes}
              ]
  }
  slider_step["args"][0]["visible"][n] = True
  slider_steps.append(slider_step)

# --- Create Figure ---
fig = go.Figure(
    data=frames,
    layout = {
        "template": "simple_white",
        "autosize": False,
        "width": 700,
        "height": 750,
        "xaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          },
        "yaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          },
        "sliders": [{
            "active": 0,
            "currentvalue": {"prefix": "Max Baseline Length (m): "},
            "pad": {"t": 50},
            "steps": slider_steps
          }]
    }
)

# --- Manualy Draw WCS Grid ---
drawWCSGrid(fig)

fig.data[0].visible = True
fig.show()

## Figure: Image & Visibilities vs Basleine Length

In [ ]:
# Generate data for each animation frame
Nframes = 16
fig = make_subplots(rows=1, cols=2)

uvmax  = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)
vismax = np.max(np.abs(vis)) * 1.05
vismin = np.min(np.abs(vis)) / 1.05

steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]

  # --- Generate Data ---
  uv_data  = UVdist[select] * (c/freq_center)
  vis_data = np.abs(vis[select])
  img_data = gen_image(select=select)

  # --- Add UVplot Trace ---
  fig.add_trace(
      go.Scattergl(
          name = "",
          x = uv_data,
          y = vis_data,
          mode = "markers",
          marker = dict(
              color = "black",
              showscale = False, # disable colorbar
              size = 4
          ),
          visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=1
  )

  fig.add_trace(
      go.Heatmap(
        name = "",
        z = img_data,
        x = l_axis,
        y = m_axis,
        colorscale = colorscale,
        visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=2
  )

  # Define slide data
  step = {
      "method": 'restyle',
      "args": ['visible', ['legendonly'] * (2*Nframes) ],
      "label": np.round(uv_cutoff*(c/freq_center),2)
  }
  step['args'][1][2*n  ] = True
  step['args'][1][2*n+1] = True
  steps.append(step)

fig.update_layout(width=1100, height=600,autosize=False)
fig.layout["template"] = "simple_white"
fig.layout["sliders"] = [{
    "steps": steps,
    "currentvalue": {"prefix": "Max Baseline Length (m): "}
}]

for n in range(Nframes):
  fig.layout["xaxis"+str(n*2+1)] = {
      "range": [0,uvmax],
      "domain": [0,0.5]
  }
  fig.layout["yaxis"+str(n*2+1)] = {
      "range": [vismin,vismax],
      "domain": [0,1]
  }

fig.data[0].visible = True
fig.data[1].visible = True

fig.show()

## Figure: Image vs UV Coverage

In [ ]:
# Generate data for each animation frame
Nframes = 12
fig = make_subplots(rows=1, cols=2)

uvmax  = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)

steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]

  # --- Generate Data ---
  UVs = UVWs[select,0:2] * (c/freq_center)
  UVs = np.tile( UVs , (2,1) )
  UVs[int(UVs.shape[0]/2):,:] *= -1

  img_data = gen_image(select=select)

  # --- Add UVplot Trace ---
  fig.add_trace(
      go.Scattergl(
          name = "",
          x = UVs[:,0],
          y = UVs[:,1],
          mode = "markers",
          marker = dict(
              color = "black",
              showscale = False, # disable colorbar
              size = 4
          ),
          visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=1
  )

  fig.add_trace(
      go.Heatmap(
        name = "",
        z = img_data,
        x = l_axis,
        y = m_axis,
        colorscale = colorscale,
        visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=2
  )

  # Define slide data
  step = {
      "method": 'restyle',
      "args": ['visible', ['legendonly'] * (2*Nframes) ],
      "label": np.round(uv_cutoff*(c/freq_center),2)
  }
  step['args'][1][2*n  ] = True
  step['args'][1][2*n+1] = True
  steps.append(step)

fig.update_layout(width=1100, height=650,autosize=False)
fig.layout["template"] = "simple_white"
fig.layout["sliders"] = [{
    "steps": steps,
    "currentvalue": {"prefix": "Max Baseline Length (m): "}
}]

for n in range(Nframes):
  fig.layout["xaxis"+str(n*2+1)] = {
      "range": [-uvmax,uvmax],
      "domain": [0,0.5]
  }
  fig.layout["yaxis"+str(n*2+1)] = {
      "range": [-uvmax,uvmax],
      "domain": [0,1]
  }

fig.data[0].visible = True
fig.data[1].visible = True

fig.show()